In [ ]:
import os

date = "mother_folder"
dl_folder = f"base_path/{date}_DL"

dl_number = "project_name"
os.mkdir(f"{dl_folder}/model_{dl_number}")

In [ ]:
experiments_list1 = [
                    "mouse1_ex1",
#                    "mouse1_ex2",
#                    "mouse1_ex3",
                    ]
mouse_id1 = "mouse1"

experiments_list2 = [
                    "mouse2_ex1",
#                    "mouse2_ex2",
#                    "mouse2_ex3",
                    ]
mouse_id2 = "mouse2"

experiments_list3 = [
                    "mouse3_ex1",
#                    "mouse3_ex2",
#                    "mouse3_ex3",
                    ]
mouse_id3 = "mouse3"


ex_list = [
            experiments_list1,
            experiments_list2, 
            experiments_list3,  
                ]

mouse_list = [
              mouse_id1,
              mouse_id2,
              mouse_id3,  
                ]

In [ ]:
fs = 20  # sampling rate
look_frame = 30     

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

def lowpass_filter(data, cutoff):

    order = 4  
    nyquist = 0.5 * fs  
    normal_cutoff = cutoff / nyquist  # normalized cut-off frequency

    # Butterworth filter
    b, a = butter(order, normal_cutoff, btype='low', analog=False)

    # applying
    filtered_data = filtfilt(b, a, data)

    return filtered_data

In [5]:
import numpy as np

def normalize(data, start_n, end_n):

    norm_data = data / np.mean(data[start_n:end_n])
    
    return norm_data

In [6]:
import numpy as np

def calc_velocity(data, dt):
     
    velocity = []
    for frame in range(data.shape[0]-1):
        vel_frame = (data[frame+1]-data[frame]) / dt
        velocity.append(vel_frame)
    velocity = np.array(velocity)

    return velocity

In [7]:
import numpy as np

def z_score(data, start_n, end_n):

    z_data = (data-np.mean(data[start_n:end_n])) / np.std(data[start_n:end_n])
    
    return z_data

In [8]:
import numpy as np

def z_score2(data, start_n, end_n):

    z_data = (data-np.mean(data[start_n:end_n])) / np.std(data[start_n:end_n])
    
    return z_data, np.mean(data[start_n:end_n]), np.std(data[start_n:end_n])

In [9]:
import numpy as np

def z_score3(data, data_base, start_n, end_n):

    z_data = (data-np.mean(data_base[start_n:end_n])) / np.std(data_base[start_n:end_n])
    
    return z_data, np.mean(data_base[start_n:end_n]), np.std(data_base[start_n:end_n])

In [10]:
import numpy as np

def min_max_norm(data):

    min_max = (data-min(data)) / (max(data)-min(data))
    
    return min_max

In [11]:
import numpy as np

def min_max_norm2(data):

    min_max = (data-min(data)) / (max(data)-min(data))
    
    return min_max, min(data), max(data)

In [12]:
import numpy as np

def calc_response(data, base_start, base_end):
    baseline = np.mean(data[base_start:base_end], axis=0)
    response = data - baseline

    return response

In [ ]:
import numpy as np
import tqdm

no_cycle = 10  # number of cycle
pupil_cutoff = 1  # cut-off frequency
vis_stim1 = 15    # stimulation start (sec)
#vis_stim2 = 25
use_range = 5     # range (sec)

diameters = []
velocities = []
accelerations = []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    diameters1= []
    velocities1= []
    accelerations1 = []
    for ex in range(len(ex_list[mouse])):
        diameter1 = []
        velocity1 = []
        acceleration1, acceleration2 = [], []
        for cycle in range(no_cycle):        
            diameter = np.load(f"pupil_folder/{mouse_list[mouse]}/{ex_list[mouse][ex]}/{ex_list[mouse][ex]}_ex0{cycle}.npy")
            diameter = lowpass_filter(data=diameter, cutoff=pupil_cutoff)
            n_diameter = normalize(data=diameter, start_n=20, end_n=90)
            velocity = calc_velocity(data=n_diameter, dt=1/fs)
            acceleration = calc_velocity(data=velocity, dt=1/fs)

            temp_dia1 = n_diameter[int(fs*vis_stim1)-int(fs*use_range):int(fs*vis_stim1)+int(fs*use_range)]
            #temp_dia2 = n_diameter[int(fs*vis_stim2)-int(fs*use_range):int(fs*vis_stim2)+int(fs*use_range)]
            temp_velo1 = velocity[int(fs*vis_stim1)-int(fs*use_range)-1:int(fs*vis_stim1)+int(fs*use_range)-1]
            #temp_velo2 = velocity[int(fs*vis_stim2)-int(fs*use_range)-1:int(fs*vis_stim2)+int(fs*use_range)-1]
            temp_acc1 = acceleration[int(fs*vis_stim1)-int(fs*use_range)-2:int(fs*vis_stim1)+int(fs*use_range)-2]
            #temp_acc2 = acceleration[int(fs*vis_stim2)-int(fs*use_range)-2:int(fs*vis_stim2)+int(fs*use_range)-2]

            diameter1.append(temp_dia1)
            #diameter2.append(temp_dia2)
            velocity1.append(temp_velo1)
            #velocity2.append(temp_velo2)
            acceleration1.append(temp_acc1)
            #acceleration2.append(temp_acc2)


        diameters1.append(np.array(diameter1))
        #diameters2.append(np.array(diameter2))
        velocities1.append(np.array(velocity1))
        #velocities2.append(np.array(velocity2))
        accelerations1.append(np.array(acceleration1))
        #accelerations2.append(np.array(acceleration2))

    #diameters.append([diameters1, diameters2])
    #velocities.append([velocities1, velocities2])
    #accelerations.append([accelerations1, accelerations2])

    diameters.append([diameters1])
    velocities.append([velocities1])
    accelerations.append([accelerations1])

#diameters = np.array(diameters)
#velocities = np.array(velocities)

print(len(diameters), len(diameters[0]), len(diameters[0][0]), diameters[0][0][0].shape)
print(len(velocities), len(velocities[0]), len(velocities[0][0]), velocities[0][0][0].shape)
print(len(accelerations), len(accelerations[0]), len(accelerations[0][0]), accelerations[0][0][0].shape)
print(len(diameters), len(diameters[1]), len(diameters[1][0]), diameters[1][0][0].shape)
print(len(velocities), len(velocities[1]), len(velocities[1][0]), velocities[1][0][0].shape)
print(len(accelerations), len(accelerations[1]), len(accelerations[1][0]), accelerations[1][0][0].shape)
print(len(diameters), len(diameters[2]), len(diameters[2][0]), diameters[2][0][0].shape)
print(len(velocities), len(velocities[2]), len(velocities[2][0]), velocities[2][0][0].shape)
print(len(accelerations), len(accelerations[2]), len(accelerations[2][0]), accelerations[2][0][0].shape)

100%|██████████| 3/3 [00:00<00:00, 11.64it/s]

3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)
3 1 4 (10, 200)


In [14]:
dia_list, vel_list, acc_list = [], [], []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    dia_array = np.array(diameters[mouse]).reshape(len(diameters[mouse])*len(diameters[mouse][0])*diameters[mouse][0][0].shape[0], diameters[mouse][0][0].shape[1])
    vel_array = np.array(velocities[mouse]).reshape(len(velocities[mouse])*len(velocities[mouse][0])*velocities[mouse][0][0].shape[0], velocities[mouse][0][0].shape[1])
    acc_array = np.array(accelerations[mouse]).reshape(len(accelerations[mouse])*len(accelerations[mouse][0])*accelerations[mouse][0][0].shape[0], accelerations[mouse][0][0].shape[1])

    dia_list.append(dia_array)
    vel_list.append(vel_array)
    acc_list.append(acc_array)

print(len(dia_list), len(vel_list), len(acc_list))
print(dia_list[0].shape, vel_list[0].shape, acc_list[0].shape)
print(dia_list[1].shape, vel_list[1].shape, acc_list[1].shape)
print(dia_list[2].shape, vel_list[2].shape, acc_list[2].shape)

100%|██████████| 3/3 [00:00<00:00, 3005.23it/s]

3 3 3
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)


In [ ]:
base_start, base_end = 85, 95  # frame

dia_res, vel_res, acc_res = [], [], []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    temp_dia, temp_vel, temp_acc = [], [], []
    for ex in range(dia_list[mouse].shape[0]):
        temp_dia.append(calc_response(dia_list[mouse][ex], base_start, base_end))
        temp_vel.append(calc_response(vel_list[mouse][ex], base_start, base_end))
        temp_acc.append(calc_response(acc_list[mouse][ex], base_start, base_end))
    dia_res.append(np.array(temp_dia))
    vel_res.append(np.array(temp_vel))
    acc_res.append(np.array(temp_acc))

print(len(dia_res), len(vel_res), len(acc_res))
print(dia_res[0].shape, vel_res[0].shape, acc_res[0].shape)
print(dia_res[1].shape, vel_res[1].shape, acc_res[1].shape)
print(dia_res[2].shape, vel_res[2].shape, acc_res[2].shape)

100%|██████████| 3/3 [00:00<00:00, 743.98it/s]

3 3 3
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)


In [ ]:
start_n, end_n = 0, 200  # frame

dia_z, vel_z, acc_z = [], [], []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    temp_dia, temp_vel, temp_acc = [], [], []
    for ex in range(dia_res[mouse].shape[0]):
        temp_dia.append(z_score(dia_res[mouse][ex], start_n, end_n))
        temp_vel.append(z_score(vel_res[mouse][ex], start_n, end_n))
        temp_acc.append(z_score(acc_res[mouse][ex], start_n, end_n))
    dia_z.append(np.array(temp_dia))
    vel_z.append(np.array(temp_vel))
    acc_z.append(np.array(temp_acc))

print(len(dia_z), len(vel_z), len(acc_z))
print(dia_z[0].shape, vel_z[0].shape, acc_z[0].shape)
print(dia_z[1].shape, vel_z[1].shape, acc_z[1].shape)
print(dia_z[2].shape, vel_z[2].shape, acc_z[2].shape)

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\user\.conda\envs\deepshap\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
100%|██████████| 3/3 [00:00<00:00, 207.55it/s]

3 3 3
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)


In [ ]:
dia_mm, vel_mm, acc_mm = [], [], []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    temp_dia, temp_vel, temp_acc = [], [], []
    for ex in range(dia_res[mouse].shape[0]):
        temp_dia.append(min_max_norm(dia_res[mouse][ex]))
        temp_vel.append(min_max_norm(vel_res[mouse][ex]))
        temp_acc.append(min_max_norm(acc_res[mouse][ex]))
    dia_mm.append(np.array(temp_dia))
    vel_mm.append(np.array(temp_vel))
    acc_mm.append(np.array(temp_acc))

print(len(dia_mm), len(vel_mm), len(acc_mm))
print(dia_mm[0].shape, vel_mm[0].shape, acc_mm[0].shape)
print(dia_mm[1].shape, vel_mm[1].shape, acc_mm[1].shape)
print(dia_mm[2].shape, vel_mm[2].shape, acc_mm[2].shape)

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\user\.conda\envs\deepshap\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
100%|██████████| 3/3 [00:00<00:00, 95.64it/s]

3 3 3
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)
(40, 200) (40, 200) (40, 200)


In [ ]:
refer_pupil = dia_mm  # selected data of pupil

In [23]:
ca_list2 = np.load("D:/Python_TK_3/datas/251126_ica/251110-251121_ss_v01/ic_temporal_components_extracted.npy")
print(ca_list2.shape)  #(models, experiments, components, frames)

(10, 119, 200)


In [25]:
ca_list2 = ca_list2.transpose(1, 0, 2)
print(ca_list2.shape)

(119, 10, 200)


In [ ]:
use_cal = [ca_list2[:refer_pupil[0].shape[0]], ca_list2[refer_pupil[0].shape[0] : refer_pupil[0].shape[0]+refer_pupil[1].shape[0]], ca_list2[refer_pupil[0].shape[0]+refer_pupil[1].shape[0] : refer_pupil[0].shape[0]+refer_pupil[1].shape[0]+refer_pupil[2].shape[0]]]

print(len(use_cal))
print(use_cal[0].shape, use_cal[1].shape, use_cal[2].shape)

3
(40, 10, 200) (40, 10, 200) (39, 10, 200)


In [ ]:
back_second = 1.0  # time before peak
forward_second = 0.5  # time after peak
stim_frame = 100  # time_frame of stimulation
pupil_thresh = 0.6

pupil_list = []
cortex_list = []
index_list =[]
corr_array = []
for mouse in tqdm.tqdm(range(len(mouse_list))):
    binary_pupil = []
    extracted_signal = []
    extracted_z_area = []
    max_idxs = []
    for ex in range(refer_pupil[mouse].shape[0]):
        max_idx = np.argmax(refer_pupil[mouse][ex, stim_frame+5:stim_frame+15]) + stim_frame+5
        extracted_pupil = refer_pupil[mouse][ex, max_idx-int(fs*back_second)-look_frame:max_idx+int(fs*forward_second)]
        use_calcium = use_cal[mouse][ex]
        extracted_ca   = use_calcium[:, max_idx-int(fs*back_second)-look_frame:max_idx+int(fs*forward_second)]

        temp_corr = []
        for comp in range(extracted_ca.shape[0]):
            temp_corr.append(np.corrcoef(extracted_pupil[look_frame:], extracted_ca[comp, look_frame:])[0, 1])

        temp_p = []
        for f in range(extracted_pupil.shape[0]):
            if extracted_pupil[f] > pupil_thresh:
                temp_p.append(1)
            else:
                temp_p.append(0)

        binary_pupil.append(temp_p)
        max_idxs.append(max_idx)
        extracted_signal.append(extracted_ca.transpose())
        corr_array.append(temp_corr)

    binary_pupil = np.array(binary_pupil)
    extracted_signal = np.array(extracted_signal)

    pupil_list.append(binary_pupil)
    cortex_list.append(extracted_signal)
    index_list.append(max_idxs)

corr_array = np.array(corr_array)

print(pupil_list[0].shape, cortex_list[0].shape, area_list[0].shape)
print(pupil_list[1].shape, cortex_list[1].shape, area_list[1].shape)
print(pupil_list[2].shape, cortex_list[2].shape, area_list[2].shape)
print(corr_array.shape)

100%|██████████| 3/3 [00:00<00:00, 17.52it/s]

(40, 60) (40, 60, 10) (40, 60, 10)
(40, 60) (40, 60, 10) (40, 60, 10)
(39, 60) (39, 60, 10) (39, 60, 10)
(119, 10)


In [31]:
mean_corr = np.mean(corr_array, axis=0)
print(mean_corr.shape)
print(np.argmax(mean_corr))
print(mean_corr[np.argmax(mean_corr)])

(10,)
5
0.0739821496075361


In [ ]:
Pupil_data = pupil_list
signal = cortex_list

peak_frame = int(fs*back_second)

target = []
feature = []
experiment_list = []
mean_array, std_array = [], []
min_array, max_array = [], []

pupil_peaks = []
cortex_peaks = []

for mouse in tqdm.tqdm(range(len(mouse_list))):
    temp_target = []
    temp_feature = []
    temp_experiment_list = []
    temp_pupil, temp_cortex = [], []
    temp_mean, temp_std = [], []
    temp_min, temp_max = [], []
    for ex in range(Pupil_data[mouse].shape[0]):
        peak_pupil = max(refer_pupil[mouse][ex])
        peak_cortex = max(area_list[mouse][ex, :, area])
        temp_pupil.append(peak_pupil)
        temp_cortex.append(peak_cortex)   
        if np.count_nonzero(Pupil_data[mouse][ex, look_frame:] == 1) > 10 and np.all(Pupil_data[mouse][ex, look_frame:look_frame+peak_frame-18] == 0):     
            temp_target.append(Pupil_data[mouse][ex])
            temp_feature.append(signal[mouse][ex].transpose())
            temp_experiment_list.append(ex)
        temp_mean.append(means[mouse][ex])
        temp_std.append(stds[mouse][ex])
        temp_min.append(mins[mouse][ex])
        temp_max.append(maxs[mouse][ex])

    target.append(np.array(temp_target))
    feature.append(np.array(temp_feature))
    experiment_list.append(np.array(temp_experiment_list))
    mean_array.append(np.array(temp_mean))
    std_array.append(np.array(temp_std))
    min_array.append(np.array(temp_min))
    max_array.append(np.array(temp_max))

    pupil_peaks.append(temp_pupil)
    cortex_peaks.append(temp_cortex)

100%|██████████| 3/3 [00:00<00:00, 231.37it/s]


In [37]:
Pupil_data = target
signal = feature
experiment_list = experiment_list

print(Pupil_data[0].shape, signal[0].shape, experiment_list[0].shape)
print(Pupil_data[1].shape, signal[1].shape, experiment_list[1].shape)
print(Pupil_data[2].shape, signal[2].shape, experiment_list[2].shape)

(17, 60) (17, 10, 60) (17,)
(13, 60) (13, 10, 60) (13,)
(12, 60) (12, 10, 60) (12,)


In [38]:
for mouse in range(len(Pupil_data)):
    print(np.sum(Pupil_data[mouse].reshape(-1))/len(Pupil_data[mouse].reshape(-1)))

0.37058823529411766
0.35512820512820514
0.34444444444444444


In [ ]:
ex_numbers0, ex_numbers1, ex_numbers2 = pupil_list[0].shape[0], pupil_list[1].shape[0], pupil_list[2].shape[0]

ex_rate = np.array([
                    experiment_list[0].shape[0]/ex_numbers0,
                    experiment_list[1].shape[0]/ex_numbers1,
                    experiment_list[2].shape[0]/ex_numbers2,
                    ])

posi_rate = np.array([
                        np.sum(Pupil_data[0].reshape(-1))/len(Pupil_data[0].reshape(-1)),
                        np.sum(Pupil_data[1].reshape(-1))/len(Pupil_data[1].reshape(-1)),
                        np.sum(Pupil_data[2].reshape(-1))/len(Pupil_data[2].reshape(-1)),
                        ])

print(ex_rate.shape, ex_rate)
print(posi_rate.shape, posi_rate)

np.save(f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_ex_rate.npy", ex_rate)
np.save(f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_positive_rate.npy", posi_rate)

(3,) [0.425      0.325      0.30769231]
(3,) [0.37058824 0.35512821 0.34444444]


In [40]:
print(experiment_list[0])
print(experiment_list[1])
print(experiment_list[2])

[ 1  4  6  8  9 10 11 13 15 18 20 22 24 25 27 30 31]
[ 0  2  3  4  6 11 17 20 27 29 30 34 37]
[ 5  7  9 11 12 13 14 16 19 20 21 24]


In [ ]:
import random


mouse = 0
random.seed(123)

# shuffle ex numbers
numbers = list(range(experiment_list[mouse].shape[0]))
random.shuffle(numbers)

# slice ex numbers array
dataset1 = numbers[0:4]
dataset2 = numbers[4:8]
dataset3 = numbers[8:11]
dataset4 = numbers[11:14]
dataset5 = numbers[14:17]

In [ ]:
import itertools


vars_lists = [dataset1, dataset2, dataset3, dataset4, dataset5]

splits1 = []
# pick up 4 groups
for group4 in itertools.combinations(vars_lists, 4):
    remaining = [v for v in vars_lists if v not in group4]
    splits1.append([
        list(group4),
        remaining,
    ])

# check
for i, pat in enumerate(splits1[:5], 1):
    four, one_a = pat
    print(f"{i:2d}:")
    print("  training data", four)
    print("  validation data→", one_a)


In [ ]:
import random


mouse = 1
random.seed(60)

# shuffle ex numbers
numbers = list(range(experiment_list[mouse].shape[0]))
random.shuffle(numbers)

# slice ex numbers array
dataset1 = numbers[0:4]
dataset2 = numbers[4:8]
dataset3 = numbers[8:11]
dataset4 = numbers[11:14]
dataset5 = numbers[14:17]

In [ ]:
import itertools


vars_lists = [dataset1, dataset2, dataset3, dataset4, dataset5]

splits1 = []
# pick up 4 groups
for group4 in itertools.combinations(vars_lists, 4):
    remaining = [v for v in vars_lists if v not in group4]
    splits1.append([
        list(group4),
        remaining,
    ])

# check
for i, pat in enumerate(splits1[:5], 1):
    four, one_a = pat
    print(f"{i:2d}:")
    print("  training data", four)
    print("  validation data→", one_a)

In [ ]:
import random


mouse = 2
random.seed(30)

# shuffle ex numbers
numbers = list(range(experiment_list[mouse].shape[0]))
random.shuffle(numbers)

# slice ex numbers array
dataset1 = numbers[0:4]
dataset2 = numbers[4:8]
dataset3 = numbers[8:11]
dataset4 = numbers[11:14]
dataset5 = numbers[14:17]

In [ ]:
import itertools


vars_lists = [dataset1, dataset2, dataset3, dataset4, dataset5]

splits1 = []
# pick up 4 groups
for group4 in itertools.combinations(vars_lists, 4):
    remaining = [v for v in vars_lists if v not in group4]
    splits1.append([
        list(group4),
        remaining,
    ])

# check
for i, pat in enumerate(splits1[:5], 1):
    four, one_a = pat
    print(f"{i:2d}:")
    print("  training data", four)
    print("  validation data→", one_a)

In [48]:
splits_list = [splits1, splits2, splits3]

In [49]:
np.save(f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_selected_experiments_mouse1.npy", np.array(experiment_list[0]))
np.save(f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_selected_experiments_mouse2.npy", np.array(experiment_list[1]))
np.save(f"{dl_folder}/model_{dl_number}/{date}_{dl_number}_selected_experiments_mouse3.npy", np.array(experiment_list[2]))

In [ ]:
import numpy

# Create input data [-n, 0]
def create_dataset(dataset, look_frame):
    dataX, dataY = [], []
    for i in range(look_frame, len(dataset)):
        xset = []
        for j in range(dataset.shape[1]-1):
            a = dataset[(i-look_frame):(i+1), j]
            xset.append(a)
        dataY.append(dataset[i, -1])      
        dataX.append(xset)
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
targetframe = 0

def Data_preprocess(signal, pupil):
    if targetframe >= 0:
        signal = numpy.array(signal).transpose()
        pupil = [[x] for x in numpy.array(pupil)]
        moved_pupil = pupil.copy()
        del moved_pupil[:targetframe]
    else:
        signal = numpy.array(signal).transpose()
        pupil = [[x] for x in numpy.array(pupil)]
        moved_pupil = pupil.copy()
        del moved_pupil[targetframe:]
    pupil =  numpy.array(moved_pupil)

    # Create dataset
    dataset = numpy.hstack([signal.astype('float32'), pupil.astype('float32')])
    X, Y = create_dataset(dataset, look_frame)

    positive_rate = pupil.sum() / len(pupil)

    return X, Y, positive_rate

In [ ]:
def multi_mouse(mouse, signal, Pupil_data, model):
    splits = splits_list[mouse]
    signal = signal[mouse]
    Pupil_data = Pupil_data[mouse]

    flat_list = [x for sublist in splits[model][0] for x in sublist]
    train_numbers = sorted(flat_list)
    valid_numbers = splits[model][1][0]

    np.save(f"{save_dir}/{date}_{dl_number}_train_numbers_mouse{mouse}.npy", np.array(train_numbers))
    np.save(f"{save_dir}/{date}_{dl_number}_valid_numbers_mouse{mouse}.npy", np.array(valid_numbers))

    TRratio_Ca = [signal[i] for i in train_numbers]
    VAratio_Ca = [signal[i] for i in valid_numbers]
    TRPupil_data = [Pupil_data[i] for i in train_numbers]
    VAPupil_data = [Pupil_data[i] for i in valid_numbers]

    TR_signal_array = np.array(TRratio_Ca).transpose(1, 0, 2)
    TR_signal_array = TR_signal_array.reshape(TR_signal_array.shape[0], TR_signal_array.shape[1]*TR_signal_array.shape[2])

    # Training dataset
    trainX = numpy.empty([0, signal.shape[1], look_frame+1], dtype=numpy.float32)
    trainY = numpy.empty(0, dtype=numpy.float32)
    pr_tr = []
    for i in list(range(len(train_numbers))):
        X, Y, pr = Data_preprocess(TRratio_Ca[i], TRPupil_data[i])
        trainX = numpy.concatenate([trainX, X], axis=0)
        trainY = numpy.concatenate([trainY, Y], axis=0)
        pr_tr.append(pr)

    # Training dataset
    validX = numpy.empty([0, signal.shape[1], look_frame+1], dtype=numpy.float32)
    validY = numpy.empty(0, dtype=numpy.float32)
    pr_va = []
    for i in list(range(len(valid_numbers))):
        X, Y, pr = Data_preprocess(VAratio_Ca[i], VAPupil_data[i])
        validX = numpy.concatenate([validX, X], axis=0)
        validY = numpy.concatenate([validY, Y], axis=0)
        pr_va.append(pr)

    # Transpose input
    input_train = trainX.transpose(0,2,1)
    input_valid = validX.transpose(0,2,1)

    posi_rate = np.array([sum(pr_tr)/len(pr_tr), sum(pr_va)/len(pr_va)])


    return input_train, input_valid, trainY, validY, TR_signal_array, posi_rate

In [ ]:
import tqdm
import numpy

no_mouse = 3
ex_frames = 30  # frames per experiment


for n in tqdm.tqdm(range(len(splits1))):
    save_dir = os.path.join(os.path.join(dl_folder, f"model_{dl_number}"), f"model_{n}")
    os.makedirs(save_dir, exist_ok=True)

    for mouse in range(no_mouse):
        temp_train, temp_valid, temp_trY, temp_vaY, temp_tr_sig, temp_posi = multi_mouse(mouse, signal, Pupil_data, model=n)
        print(temp_posi)
        if mouse == 0:
            input_train, input_valid, trainY, validY, TR_signal_array  = temp_train, temp_valid, temp_trY, temp_vaY, temp_tr_sig
        else:
            input_train = np.concatenate((input_train, temp_train), axis=0)
            input_valid = np.concatenate((input_valid, temp_valid), axis=0)
            trainY = np.concatenate((trainY, temp_trY), axis=0)
            validY = np.concatenate((validY, temp_vaY), axis=0)
            TR_signal_array = np.concatenate((TR_signal_array, temp_tr_sig), axis=1)



    signal_min = []
    signal_max = []
    signal_ave = []
    signal_std = []
    for area in range(TR_signal_array.shape[0]):
        signal_min.append(min(TR_signal_array[area]))
        signal_max.append(max(TR_signal_array[area]))
        signal_ave.append(np.mean(TR_signal_array[area]))
        signal_std.append(np.std(TR_signal_array[area]))
    signal_min = np.array(signal_min)
    signal_max = np.array(signal_max)
    signal_ave = np.array(signal_ave)
    signal_std = np.array(signal_std)

    np.save(f"{save_dir}/{date}_{dl_number}_train_signal_min.npy", signal_min)
    np.save(f"{save_dir}/{date}_{dl_number}_train_signal_max.npy", signal_max)
    np.save(f"{save_dir}/{date}_{dl_number}_train_signal_ave.npy", signal_ave)
    np.save(f"{save_dir}/{date}_{dl_number}_train_signal_std.npy", signal_std)

    # shuffle train y labels
    #ex_no = int(input_train.shape[0]/ex_frames)
    #for ex in range(ex_no):
    #    copy_array = trainY[ex*ex_frames:(ex+1)*ex_frames].copy()
    #    np.random.shuffle(copy_array)
    #    trainY[ex*ex_frames:(ex+1)*ex_frames] = copy_array

    np.save(f"{save_dir}/{date}_{dl_number}_train_features.npy", input_train)
    np.save(f"{save_dir}/{date}_{dl_number}_train_targets.npy", trainY)
    np.save(f"{save_dir}/{date}_{dl_number}_valid_features.npy", input_valid)
    np.save(f"{save_dir}/{date}_{dl_number}_valid_targets.npy", validY)

 40%|████      | 2/5 [00:00<00:00, 18.47it/s]

[0.39285714 0.26666667]
[0.33636364 0.45833333]
[0.37333333 0.2       ]
[0.33095238 0.55555556]
[0.33939394 0.44166667]
[0.35666667 0.28333333]
[0.3702381  0.37222222]
[0.36666667 0.31666667]
[0.36       0.26666667]
[0.40512821 0.25833333]
[0.37166667 0.3       ]
[0.30925926 0.45      ]


100%|██████████| 5/5 [00:00<00:00, 19.28it/s]

[0.35512821 0.42083333]
[0.365      0.32222222]
[0.31666667 0.42777778]
